In [96]:
print("OK")

OK


In [97]:
%pwd

'/Users'

In [98]:
import os
os.chdir("../")

In [99]:
%pwd

'/'

In [100]:
import sys
print(sys.executable)


/usr/local/bin/python3


In [101]:
!pip show langchain


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: langchain
Version: 0.3.25
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [102]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [103]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [ ]:
import os

# Step 1: Change to your project directory
os.chdir('/Users/macbookpro/Downloads/Medical-Chatbot')

# Step 2: Confirm working directory
print("Current working directory:", os.getcwd())
print("Contents:", os.listdir())

# Step 3: Load the PDFs from the Data/ folder
extracted_data = load_pdf_file(data='Data/')


Current working directory: /Users/macbookpro/Downloads/Medical-Chatbot
Contents: ['research', '.DS_Store', 'LICENSE', 'requirements.txt', 'langchain_env', 'README.md', 'setup.py', '.gitignore', '.env', 'app.py', 'template.py', 'Generative_AI_Project.egg-info', '.git', 'Data', 'src']


In [ ]:
#extracted_data

In [ ]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks


In [ ]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))

190


In [34]:
from langchain.embeddings import HuggingFaceEmbeddings

In [35]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [39]:
import sys
print(sys.executable)


/usr/local/bin/python3


In [40]:
embeddings=download_hugging_face_embeddings()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
query_result=embeddings.embed_query("Hello World")
print(len(query_result))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


384


In [43]:
#query_result

In [48]:
from dotenv import load_dotenv
load_dotenv()

True

In [49]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [61]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_5wzny3_9S9syrVoVo6iYtEAih93NCYseGWHjmJUCx72bXqYCYPtTrfDeFQDceXL1mA28Xb")

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)


{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-4jwo93a.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [67]:
from dotenv import load_dotenv
load_dotenv()


True

In [68]:
import os
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")


In [70]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_5wzny3_9S9syrVoVo6iYtEAih93NCYseGWHjmJUCx72bXqYCYPtTrfDeFQDceXL1mA28Xb")


In [76]:
import os
os.environ["PINECONE_API_KEY"]="pcsk_5wzny3_9S9syrVoVo6iYtEAih93NCYseGWHjmJUCx72bXqYCYPtTrfDeFQDceXL1mA28Xb"

In [77]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
